In [1]:
from transformers import ViTFeatureExtractor

feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch32-384")

In [5]:
import os
from PIL import image
from tqdm import tqdm

images = []
for filename in tqdm(os.listdir("data/shuffled_square_75")[:300]):
    file = f"data/shuffled_square_75/{filename}"
    img = image.open(file)
    images.append(img)

images_train = images[:100]
images_val = images[100:200]
images_test = images[200:]

ImportError: cannot import name 'image' from 'PIL' (c:\Users\Valen\Anaconda3\envs\airogs\lib\site-packages\PIL\__init__.py)

In [3]:
from torchvision.transforms import (Compose, 
                                    Normalize, 
                                    RandomHorizontalFlip,
                                    RandomVerticalFlip,
                                    GaussianBlur,
                                    RandomRotation, #  rotation
                                    Resize, 
                                    ToTensor)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
_train_transforms = Compose(
        [
            Resize(feature_extractor.size),
            RandomHorizontalFlip(),
            RandomVerticalFlip(),
            RandomRotation(45),
            GaussianBlur(5, (0.1, 0.2)),
            ToTensor(),
            normalize,
        ]
    )

_val_transforms = Compose(
        [
            Resize(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(images):
    return [_train_transforms(image) for img in tqdm(images)]

def val_transforms(images):
    return [_val_transforms(image) for img in tqdm(images)]

In [4]:
train_transformed = train_transforms(images_train)
train_transformed[0]

TypeError: img should be PIL Image. Got <class 'numpy.ndarray'>

In [ ]:
from torch.utils.data import DataLoader
import torch

def collate_fn(images)):
    pixel_values = torch.stack(images)
    labels = torch.tensor(labels)
    return pixel_values, labels

train_batch_size = 2
eval_batch_size = 2

train_dataloader = DataLoader(train_ds, shuffle=True, collate_fn=collate_fn, batch_size=train_batch_size)
val_dataloader = DataLoader(val_ds, collate_fn=collate_fn, batch_size=eval_batch_size)
test_dataloader = DataLoader(test_ds, collate_fn=collate_fn, batch_size=eval_batch_size)